In [1]:
#install gradio

!pip install gradio==4.44.0

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# ────────────────── 导入相关包 ──────────────────
import os
import base64
import time
import gradio as gr
import mindspore
from mindspore import context

from mindnlp.transformers import (
    Qwen2VLForConditionalGeneration,
    AutoProcessor,
)
from qwen_vl_utils import process_vision_info

[WARNING] ME(4916:255086446710816,MainProcess):2025-08-06-20:20:55.592.653 [mindspore/run_check/_check_version.py:324] MindSpore version 2.5.0 and Ascend AI software package (Ascend Data Center Solution)version 8.2 does not match, the version of software package expect one of ['7.5', '7.6']. Please refer to the match info on: https://www.mindspore.cn/install
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, ge

In [3]:
# ────────────────── MindSpore 环境 ──────────────────
context.set_context(enable_debug_runtime=True)
context.set_context(mode=context.GRAPH_MODE, max_call_depth=2000)
print("Device target:", mindspore.get_context("device_target"))

[WARNING] ME(4916:255086446710816,MainProcess):2025-08-06-20:21:46.980.094 [mindspore/context.py:1335] For 'context.set_context', the parameter 'max_call_depth' will be deprecated and removed in a future version. Please use the api mindspore.set_recursion_limit() instead.


Device target: Ascend


In [4]:
# ────────────────── 模型与处理器 ──────────────────
MODEL_PATH = "/home/HwHiAiUser/.cache/modelscope/hub/models/Qwen/Qwen2-VL-2B-Instruct"
FP16 = mindspore.float16

print("Loading model ... (首次可能较慢)")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    ms_dtype=FP16,
    trust_remote_code=True,
)

min_pixels = 256 * 28 * 28
max_pixels = 1024 * 28 * 28
processor = AutoProcessor.from_pretrained(
    MODEL_PATH,
    min_pixels=min_pixels,
    max_pixels=max_pixels,
    ms_dtype=FP16,
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'type', 'mrope_section'}
Qwen2VLForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading model ... (首次可能较慢)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MindSpore do not support bfloat16 dtype, we will automaticlly convert to float16


In [18]:

import io
import os
import time
import json
import base64
import random

import gradio as gr
from PIL import Image
from pathlib import Path
from typing import List, Dict

from mindspore import Tensor
# ─────────── 工具函数 ───────────

def pil_to_data_uri(pil_img):
    buf = io.BytesIO()
    pil_img.save(buf, format="PNG")
    b64 = base64.b64encode(buf.getvalue()).decode()
    return f"data:image/png;base64,{b64}"

def history_to_messages(hist: List[Dict]):
    """our state(list[dict]) → Qwen2-VL messages"""
    msgs = []
    for turn in hist:
        user = {"role": "user", "content": []}
        if turn["user_img"]:
            user["content"].append({"type": "image", "image": turn["user_img"]})
        if turn["user_text"].strip():
            user["content"].append({"type": "text", "text": turn["user_text"]})
        msgs.append(user)
        msgs.append({"role": "assistant", "content": turn["assistant"]})
    return msgs

def generate_reply(hist: List[Dict]):
    msgs = history_to_messages(hist)
    prompt = processor.apply_chat_template(
        msgs, tokenize=False, add_generation_prompt=True
    )
    image_inputs, _ = process_vision_info(msgs)

    inputs = processor(
        text=[prompt], images=image_inputs,
        padding=True, return_tensors="ms"
    )

    for k in ("input_ids", "attention_mask", "position_ids"):
        if k in inputs and inputs[k].dtype == mindspore.int64:
            inputs[k] = inputs[k].astype(mindspore.int32)

    outs = model.generate(**inputs, max_new_tokens=256)
    trimmed = [o[len(i):] for i, o in zip(inputs["input_ids"], outs)]
    text = processor.batch_decode(
        trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )[0].strip()
    return text

def ui_history(hist: List[Dict]):
    """state → Chatbot 所需 [(user, assistant), ...]"""
    ui = []
    for t in hist:
        u_disp = ""
        if t["user_img"]:
            img = Image.open(t["user_img"])
            u_disp += f'<img src="{pil_to_data_uri(img)}" width="128"/> '
        u_disp += t["user_text"]
        ui.append((u_disp, t["assistant"]))
    return ui

# ─────────── Gradio 回调 ───────────
def chat_fn(user_text, user_img, state):
    """
    Parameters
    ----------
    user_text : str
    user_img  : PIL.Image | None
    state     : list[dict]
    Returns (generator)
    -------
    chatbot_history, state
    """
    state = state or []

    # 保存图片到本地，路径传给模型
    img_path = None
    if user_img is not None:
        os.makedirs("tmp_uploads", exist_ok=True)
        img_path = f"tmp_uploads/{int(time.time() * 1000)}.png"
        user_img.save(img_path)

    # 添加新 turn，assistant 先留空
    state.append({
        "user_text": user_text or "",
        "user_img": img_path,
        "assistant": ""
    })

    # 第一次 yield：先把用户消息展示出来
    yield ui_history(state), state

    # 模型推理
    reply = generate_reply(state)
    state[-1]["assistant"] = reply

    # 第二次 yield：展示完整对话
    yield ui_history(state), state


In [22]:

# ─────────── Gradio 组件定义 ───────────
textbox = gr.Textbox(lines=2, placeholder="请输入文字…", label="问题")
image = gr.Image(type="pil", label="上传图片（可选）")
state = gr.State([])  # 隐藏的对话上下文
chatbot = gr.Chatbot(height=480,
                     bubble_full_width=False,
                     label="对话")

# ─────────── Gradio Interface ───────────
demo = gr.Interface(
    fn=chat_fn,
    inputs=[textbox, image, state],
    outputs=[chatbot, state],
    title="Qwen2-VL MindSpore Demo",
    description="上传图片或输入文本，与 Qwen2-VL 进行多模态对话。",
    theme="soft",
    allow_flagging="never",
    clear_btn="清空",
    concurrency_limit=4
)

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7868, share=False)

Running on local URL:  http://0.0.0.0:7868

To create a public link, set `share=True` in `launch()`.
